<a href="https://colab.research.google.com/github/saintblue97/DeepNN_Optiver/blob/main/brett_10_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive; drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
#!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v optiver-trading-at-the-close        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
# !kaggle competitions leaderboard --show                       # print public leaderboard

- competition is now set to: optiver-trading-at-the-close
100% 201M/201M [00:10<00:00, 19.5MB/s]


In [3]:
%%time
%%capture
%reset -f
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, xgboost, lightgbm as lgb, librosa

ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=300): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 1.62 s, sys: 387 ms, total: 2.01 s
Wall time: 4.01 s


In [4]:
def is_google_colab():
    """Check if the environment is Google Colab.

    Returns:
        bool: True if in Google Colab, False otherwise.
    """
    try:
        import google.colab
        return True
    except ImportError:
      return False

if is_google_colab():
  file_path = ''
else:
  file_path = '/kaggle/input/optiver-trading-at-the-close/'

In [5]:
sample_submission_path = file_path + 'example_test_files/sample_submission.csv'

sample_submission = pd.read_csv(sample_submission_path); sample_submission

,time_id,row_id,target
0,26290,478_0_0,0
1,26290,478_0_1,0
...,...,...,...
32998,26454,480_540_198,0
32999,26454,480_540_199,0


# Load Data

In [6]:
tmr = Timer()

⏳ started. You have 300 sec. Good luck!


In [7]:
train_data_path = file_path + 'train.csv'
df = pd.read_csv(train_data_path);
df = df.dropna(subset=['target']); df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
1,1,0,0,1.67e+05,-1,1.0,1.64e+06,NaN,NaN,1.0,3233.04,1.0,20605.09,1.0,-5.52,0,0_0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237978,198,480,540,1.00e+06,1,1.0,9.48e+07,1.0,1.0,1.0,125631.72,1.0,669893.00,1.0,-1.54,26454,480_540_198
5237979,199,480,540,1.88e+06,-1,1.0,2.41e+07,1.0,1.0,1.0,250081.44,1.0,300167.56,1.0,-6.53,26454,480_540_199


In [8]:
test_data_path = file_path + 'example_test_files/test.csv'
df_test  = pd.read_csv(test_data_path); df_test

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,currently_scored
0,0,478,0,3.75e+06,-1,1.0,1.15e+07,NaN,NaN,1.0,22940.00,1.0,9177.60,1.0,26290,478_0_0,False
1,1,478,0,9.86e+05,-1,1.0,3.85e+06,NaN,NaN,1.0,1967.90,1.0,19692.00,1.0,26290,478_0_1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32998,198,480,540,1.00e+06,1,1.0,9.48e+07,1.0,1.0,1.0,125631.72,1.0,669893.00,1.0,26454,480_540_198,False
32999,199,480,540,1.88e+06,-1,1.0,2.41e+07,1.0,1.0,1.0,250081.44,1.0,300167.56,1.0,26454,480_540_199,False


In [9]:
print('Shape of training data =', df.shape)
print('Shape of testing data  =', df_test.shape)

Shape of training data = (5237892, 17)
Shape of testing data  = (33000, 17)


In [10]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler,OneHotEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import ARDRegression, BayesianRidge, LinearRegression, Ridge, Lars
from itertools import combinations
import gc
import os
import time

In [11]:
# n_val = 50000

tX, tY = df.drop('target', axis=1), df.target    # full training set

# X_train = tX
# y_train = tY

# from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(tX, tY, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1)

# Preprocessing

In [12]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Only drop columns that exist in X
        columns_to_drop = [col for col in self.columns_to_drop if col in X.columns]
        return X.drop(columns_to_drop, axis=1)

class DataFrameSimpleImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean', fill_value=None):
        self.strategy_ = strategy
        self.fill_value_ = fill_value

    def fit(self, X, y=None):
        if self.strategy_ == 'mean':
            self.fill_values_ = X.mean()
        elif self.strategy_ == 'median':
            self.fill_values_ = X.median()
        elif self.strategy_ == 'constant':
            if self.fill_value_ is None:
                raise ValueError("fill_value must be provided for strategy='constant'")
            self.fill_values_ = pd.Series(self.fill_value_, index=X.columns)
        else:
            raise ValueError(f"Unknown strategy type: {self.strategy_}")

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill_values_)

class MovingAverageTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, window_size, columns):
        self.window_size = window_size  # The size of the window for moving average
        self.columns = columns          # The columns on which to apply moving average

    def fit(self, X, y=None):
        return self  # No fitting process, so just return self

    def calculate_moving_average(self, series):
        return series.rolling(window=self.window_size).mean()

    def transform(self, X):
        X_new = X.copy()

        for col in self.columns:
            X_new[f'{col}_sma_{self.window_size}'] = X_new.groupby('stock_id')[col].transform(self.calculate_moving_average)

            X_new[f'{col}_sma_{self.window_size}'].fillna(X_new[col], inplace=True)


        return X_new


In [13]:
# generate imbalance features
import numpy as np
import pandas as pd
from itertools import combinations

def imbalance_features(df):
    df = df.copy()
    prices = ["reference_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size"]

    df.loc[:, "volume"] = df["ask_size"] + df["bid_size"]
    df.loc[:, "mid_price"] = (df["ask_price"] + df["bid_price"]) / 2
    df.loc[:, "liquidity_imbalance"] = (df["bid_size"] - df["ask_size"]) / (df["bid_size"] + df["ask_size"])

    for c in combinations(prices, 2):
        df.loc[:, f"{c[0]}_{c[1]}_imb"] = (df[c[0]] - df[c[1]]) / (df[c[0]] + df[c[1]])

    df.loc[:, "stock_weights"] = df["stock_id"].map(weights_dict)
    df.loc[:, "weighted_wap"] = df["stock_weights"] * df["wap"]
    df.loc[:, "price_spread"] = df["ask_price"] - df["bid_price"]
    df.loc[:, "spread_depth_ratio"] = df["price_spread"] / (df['bid_size'] + df['ask_size'])

    df.loc[:, f"all_prices_mean"] = df[prices].mean(axis=1)
    df.loc[:, f"all_prices_std"] = df[prices].std(axis=1)
    df.loc[:, f"all_sizes_mean"] = df[sizes].mean(axis=1)
    df.loc[:, f"all_sizes_std"] = df[sizes].std(axis=1)

    return df



def other_features(df):
    df = df.copy()
    df.loc[:, "dow"] = df["date_id"] % 5
    df.loc[:, "dom"] = df["date_id"] % 20
    df.loc[:, "seconds"] = df["seconds_in_bucket"] % 60
    df.loc[:, "minute"] = df["seconds_in_bucket"] // 60

    for key, value in global_stock_id_feats.items():
        df.loc[:, f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

# generate all features
def generate_all_features(df):
    df = df.copy()
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
    df = df[cols]
    df = imbalance_features(df)
    df = other_features(df)
    gc.collect()

    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]

    return df[feature_name]

def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in range(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [14]:
class ImbalanceFeaturesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # No fitting process needed

    def transform(self, X):
        # Make sure that imbalance_features returns a DataFrame
        transformed_X = imbalance_features(X)
        return transformed_X


class OtherFeaturesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # No fitting process needed

    def transform(self, X):
        return other_features(X)


class AllFeaturesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # No fitting process needed

    def transform(self, X):
        return generate_all_features(X)


In [15]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

global_stock_id_feats = {
        "median_size": df.groupby("stock_id")["bid_size"].median() + df.groupby("stock_id")["ask_size"].median(),
        "std_size": df.groupby("stock_id")["bid_size"].std() + df.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df.groupby("stock_id")["bid_size"].max() - df.groupby("stock_id")["bid_size"].min(),
        "median_price": df.groupby("stock_id")["bid_price"].median() + df.groupby("stock_id")["ask_price"].median(),
        "std_price": df.groupby("stock_id")["bid_price"].std() + df.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df.groupby("stock_id")["bid_price"].max() - df.groupby("stock_id")["ask_price"].min(),
    }

In [16]:
# Convert weights list to a dictionary
stock_ids = range(len(weights))  # Replace this with your actual stock IDs if they are different
weights_dict = dict(zip(stock_ids, weights))

In [17]:
from sklearn.compose import ColumnTransformer

# Set up preprocessing pipeline
preprocess_steps = [
    ('drop_columns', DropColumns(['time_id', 'currently_scored'])),
    ('imputer1', DataFrameSimpleImputer(strategy='constant', fill_value=0)),
    ('moving_average', MovingAverageTransformer(window_size=2, columns=['ask_size', 'ask_price', 'bid_price', 'ask_size', 'reference_price', 'wap'])),
    ('imbalance_features', ImbalanceFeaturesTransformer()),
    ('other_features', OtherFeaturesTransformer()),
    ('all_features', AllFeaturesTransformer()),
    ('imputer2', DataFrameSimpleImputer(strategy='constant', fill_value=0)),
]
pipeline = Pipeline(preprocess_steps)

In [18]:
X_transformed = pipeline.fit_transform(X_train)
X_transformed

,stock_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,...,dow,dom,seconds,minute,global_median_size,global_std_size,global_ptp_size,global_median_price,global_std_price,global_ptp_price
3006091,122,0,1.31e+07,1,1.0,9.29e+06,0.0,0.0,1.0,653001.20,...,3,18,0,0,485814.63,528909.11,5.73e+06,2.0,4.30e-03,0.02
919317,95,180,4.87e+07,1,1.0,9.88e+07,0.0,0.0,1.0,14273.00,...,1,6,0,3,82795.09,439038.50,7.29e+06,2.0,5.33e-03,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4886930,150,40,1.50e+07,-1,1.0,2.31e+07,0.0,0.0,1.0,96288.62,...,4,9,40,0,48222.17,148235.47,3.32e+06,2.0,4.39e-03,0.02
4946751,171,280,5.38e+06,-1,1.0,7.72e+06,0.0,0.0,1.0,8398.89,...,4,14,40,4,30478.50,70335.52,9.38e+05,2.0,3.93e-03,0.02


# Make submission

In [19]:
# from sklearn.metrics import make_scorer,mean_absolute_error
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, PassiveAggressiveClassifier, Perceptron, RidgeClassifier, RidgeClassifierCV, SGDClassifier, SGDOneClassSVM
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# if is_google_colab():

#     models = [
#         LinearRegression(),
#         # Ridge(),
#         # BayesianRidge(),
#     ]


#     for model in models:
#         model_pipeline = Pipeline(preprocess_steps + [('model', model)])

#         scores = cross_val_score(model_pipeline, tX.copy(), tY.copy(), cv=5, scoring='neg_mean_absolute_error')

#         print(f'model={model} | score={np.mean(scores):.3f}{chr(177)} {np.std(scores):.3f}\n')

In [20]:
# Train enseamble of models

models = {
        'lr': LinearRegression(),
        'lgb': lgb.LGBMRegressor(objective='regression_l1', n_estimators=250),
#         'xgb': xgb.XGBRegressor(tree_method='hist', objective='reg:absoluteerror', n_estimators=250)
}
for name, model in models.items():
    print(f'Training {name} model')

    model.fit(
        X_transformed,
        y_train.values.ravel())

Training lr model
Training lgb model
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.175315 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9386
[LightGBM] [Info] Number of data points in the train set: 4242691, number of used features: 45
[LightGBM] [Info] Start training from score -0.060201


In [21]:
# if not is_google_colab():
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [22]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

if is_google_colab():

    # Lists to store true and predicted targets
    true_targets = []
    predicted_targets = []

    for (test, revealed_targets, sample_prediction) in iter_test:
        # Predict target values
        test_processed = pipeline.transform(test)
        predictions = np.mean([model.predict(test_processed) for _, model in models.items()], 0)

        # Only take as many predictions and true targets as the minimum of the two
        min_len = min(len(predictions), len(revealed_targets))

        # Trim down the predictions list so that it is the same size as the revealed targets
        current_predictions = predictions[:min_len]
        current_true_targets = revealed_targets['revealed_target'].values[:min_len]

        # Extend the lists of true targets and predictions with the ones from the current batch
        true_targets.extend(current_true_targets)
        predicted_targets.extend(current_predictions)

        # Make the actual prediction and add to sample_prediction df
        sample_prediction['target'] = predictions

        # Submit the predictions for this chunk of test data
        env.predict(sample_prediction)

    # Convert lists to dataframes
    df_true = pd.DataFrame(true_targets, columns=['true_target'])
    df_pred = pd.DataFrame(predicted_targets, columns=['predicted_target'])

    # Calculate and print the MAE
    mae = mean_absolute_error(df_true.fillna(0), df_pred)
    print("Mean Absolute Error:", mae)
    # 5.261823197997517

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Mean Absolute Error: 5.563923515679605


In [23]:
if not is_google_colab():
  counter = 0
  for (test, revealed_targets, sample_prediction) in iter_test:
      test_processed = pipeline.transform(test).values
      sample_prediction['target'] = np.mean([model.predict(test_processed) for _, model in models.items()], 0)
      env.predict(sample_prediction)
      counter += 1

In [24]:
tmr.ShowTime()    # measure Colab's runtime.

Runtime is 396 sec > 300 sec limit!!!
